**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../config/config_sing.R")))
show_env()

You are in Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/out 
CODE DIRECTORY (FD_CODE): /data/reddylab/Kuei/code 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/code/Proj_CombEffect_ENCODE_FCC 
PATH OF RESULTS (FD_RES): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc 
PATH OF LOG     (FD_LOG): /data/reddylab/Kuei/out/proj_combeffect_encode_fcc/log 


**Check data**

In [2]:
fdiry = file.path(FD_RES, "results")
dir(fdiry)

[1] "A001_K562_WSTARRseq"          "comparison"                  
 [3] "CRISPRi_FlowFISH"             "CRISPRi_Growth_K562_Gersbach"
 [5] "hic_intact_K562_deep"         "hic_intact_K562_ENCSR479XDG" 
 [7] "KS91_K562_ASTARRseq"          "LentiMPRA_K562_Nadav"        
 [9] "region"                       "Tewhey_K562_TileMPRA"

In [3]:
fdiry = file.path(FD_RES, "results", "region", "KS91_K562_ASTARRseq_peak_macs_input")
print(dir(fdiry))

 [1] "annotation_ccre"                                                          
 [2] "annotation_chipseq_histone"                                               
 [3] "annotation_chipseq_tf"                                                    
 [4] "annotation_chipseq_tf_subset"                                             
 [5] "annotation_chromHMM"                                                      
 [6] "annotation_crispri_hcrff"                                                 
 [7] "annotation_enhancer_junke"                                                
 [8] "annotation_enhancer_zscore_junke"                                         
 [9] "annotation_hic"                                                           
[10] "annotation_tss_pol2"                                                      
[11] "annotation_tss_pol2_span"                                                 
[12] "description.tsv"                                                          
[13] "enrichment_prepare_ann

## Import data

In [5]:
### init
ASSAYS = c("KS91_K562_ASTARRseq", "A001_K562_WSTARRseq", "Tewhey_K562_TileMPRA")
LABELS = c("ASTARR", "WSTARR", "TMPRA")
FOLDER = "coverage_astarrseq_peak_macs_input"

fnames = c(
    "result.Log2FC.raw.deseq.WGS.tsv",
    "result.Log2FC.raw.deseq.WGS.tsv",
    "result.Log2FC.raw.deseq.Merge.tsv"
)
names(fnames) = ASSAYS

### loop and import data
lst = lapply(ASSAYS, function(ASSAY){
    
    ### set file directories
    fdiry = file.path(FD_RES, "results", ASSAY, FOLDER, "summary")
    fname = fnames[ASSAY]
    fpath = file.path(fdiry, fname)

    ### read data
    dat = read_tsv(fpath, show_col_types = FALSE)
    return(dat)
})

names(lst) = LABELS
dat = bind_rows(lst, .id = "Assay")

### assign and show
dat_peak_deseq = dat
print(dim(dat))
head(dat)

[1] 495260      9


Assay,Peak,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Dataset
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
ASTARR,chr1:100006256-100006880,74.39238,-0.01311792,0.15512423,-0.08456397,9.326080e-01,0.9510236543,NA
ASTARR,chr1:100010437-100010915,48.25931,-0.80687947,0.22174651,-3.63874709,2.739677e-04,0.0008947519,NA
ASTARR,chr1:10002087-10003910,224.35927,-0.34892859,0.09743886,-3.58100041,3.422811e-04,0.0010926173,NA
ASTARR,chr1:100021298-100021629,38.13434,-1.03799571,0.25401013,-4.08643436,4.380534e-05,0.0001707736,NA
ASTARR,chr1:100023727-100023976,32.62049,-0.19100207,0.25625467,-0.74536036,4.560539e-01,0.5497617125,NA
ASTARR,chr1:100027983-100029702,290.98442,-0.18001582,0.09610434,-1.87312902,6.105057e-02,0.1032752224,NA


## Arrange

In [6]:
dat = dat_peak_deseq
dat = dat %>%
    dplyr::select(Assay, Peak, log2FoldChange) %>%
    tidyr::spread(Assay, log2FoldChange) %>%
    na.omit

### assign and show
dat_peak_log2fc = dat
print(dim(dat))
head(dat)

[1] 1722    4


Peak,ASTARR,TMPRA,WSTARR
<chr>,<dbl>,<dbl>,<dbl>
chr11:32870601-32871324,-0.56726545,-0.7592992,-0.45442671
chr11:32874343-32875070,-0.54920178,-0.4159155,-0.23994680
chr11:32884749-32885822,-0.66011484,0.2420861,-0.17415901
chr11:32892099-32894437,-0.04192326,0.9418955,0.42636977
chr11:32901572-32902485,-0.38438377,-0.8541150,-0.09841904
chr11:32903491-32904506,-0.05446687,-0.7215339,-0.68078185


## save table

In [7]:
### set file directories
fdiry = file.path(FD_RES, "results", "comparison")
fname = "result.Log2FC.raw.deseq.starrmpra.tsv"
fpath = file.path(fdiry, fname)
print(fpath)

write_tsv(dat_peak_log2fc, fpath)

[1] "/data/reddylab/Kuei/out/proj_combeffect_encode_fcc/results/comparison/result.Log2FC.raw.deseq.starrmpra.tsv"
